<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/Long_coder_1_n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW





In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MSR_1vs1_23000.csv')

In [ ]:
df = df.drop(columns="CWE ID")
df = df.drop(columns="CVE ID")
df = df.drop(columns="vul")


In [ ]:
df.columns

Index(['CODE', 'lang', 'IDX', 'chmod()', 'assert()', 'atoi()', 'atof()',
       'calloc()', 'errno()', 'exit()',
       ...
       'array_index_vulnerability', 'int', 'short', 'long', 'long long',
       'unsigned int', 'unsigned short', 'unsigned long', 'unsigned long long',
       'line_count'],
      dtype='object', length=103)

In [ ]:
import random
num_rows = len(df)
random_indexes = random.sample(range(num_rows), 100)
df = df.drop(random_indexes)
test = df.iloc[random_indexes]
test

,CODE,lang,IDX,chmod(),assert(),atoi(),atof(),calloc(),errno(),exit(),...,array_index_vulnerability,int,short,long,long long,unsigned int,unsigned short,unsigned long,unsigned long long,line_count
4228,static inline enum iucv_tx_notify qeth_compute...,C,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,24
21711,"void Browser::FindInPage(bool find_next, bool ...",C,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
15643,virtual InputMethodDescriptor previous_input...,C,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
18095,static void php_snmp(INTERNAL_FUNCTION_PARAMET...,C,3,0,0,0,0,0,0,0,...,0,5,0,2,0,0,0,0,0,139
5434,"void Gfx::opSetLineCap(Object args[], int numA...",CPP,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3070,static void ssd0323_class_init(ObjectClass *kl...,C,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
7782,"int kvm_get_msr(struct kvm_vcpu *vcpu, u32 msr...",C,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
5474,"XRecordGetContext(Display *dpy, XRecordContext...",C,0,0,0,0,0,2,0,0,...,0,1,0,2,0,1,0,0,0,104
13372,PHP_FUNCTION(locale_get_display_language) \n {...,C,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [ ]:
from transformers import LongformerConfig, RobertaTokenizer, pipeline
tokenizer = RobertaTokenizer.from_pretrained("microsoft/longcoder-base")
model = RobertaForSequenceClassification.from_pretrained("microsoft/longcoder-base",num_labels = 10)
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

You are using a model of type longformer to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/longcoder-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)

def tokenized(examples):
  return tokenizer(examples['CODE'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)


In [ ]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='IDX',new_column_name='labels')


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/23732 [00:00<?, ? examples/s]

In [ ]:
preprocess(test,"preprocess_test.csv")
test_dataset = load_dataset("csv",data_files="preprocess_test.csv")
encoded_test_dataset = test_dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_test_dataset=encoded_test_dataset.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=100)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = load_metric("accuracy")
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')  # 클래스 불균형을 고려한 정밀도
    recall = recall_score(labels, predictions, average='weighted')  # 클래스 불균형을 고려한 재현율
    f1 = f1_score(labels, predictions, average='weighted')  # 클래스 불균형을 고려한 F1 스코어
    accuracy = accuracy_score(labels, predictions)  # 정확도 계산
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',             # 학습 결과를 저장할 디렉터리
    num_train_epochs=7,                 # 데이터셋 크기에 맞춰 조정된 학습 에폭 수
    per_device_train_batch_size=16,     # 훈련 배치 크기, GPU 메모리에 맞게 조정 가능
    per_device_eval_batch_size=64,      # 평가 배치 크기, GPU 메모리에 맞게 조정 가능
    warmup_steps=500,                   # 워밍업 스텝 수, 총 스텝의 10%를 고려하여 조정 가능
    weight_decay=0.01,                  # 가중치 감소
    evaluation_strategy="epoch",        # 에폭마다 평가 수행
    logging_dir='./logs',               # 로그를 저장할 디렉터리
)

trainer = Trainer(
    model=model,                         # 학습시킬 모델
    args=training_args,                  # 학습 인자
    train_dataset=encoded_dataset['train'],         # 훈련 데이터셋
    eval_dataset=encoded_dataset['test'],           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 메트릭 계산 함수
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.902800,0.729345,0.763764,0.798562,0.763764,0.743869
2,0.520400,0.549984,0.835253,0.834363,0.835253,0.829146
3,0.295800,0.501203,0.875562,0.885982,0.875562,0.877302
4,0.167400,0.467540,0.906601,0.906197,0.906601,0.905420
5,0.110900,0.483798,0.911376,0.912077,0.911376,0.911442
6,0.066800,0.491449,0.921067,0.921608,0.921067,0.921072
7,0.041100,0.493903,0.923034,0.922908,0.923034,0.922809


TrainOutput(global_step=7273, training_loss=0.33623245456401313, metrics={'train_runtime': 3119.6701, 'train_samples_per_second': 37.274, 'train_steps_per_second': 2.331, 'total_flos': 3.0597803606286336e+16, 'train_loss': 0.33623245456401313, 'epoch': 7.0})

In [ ]:
trainer.test()

AttributeError: 'Trainer' object has no attribute 'test'